In [ ]:
import numpy as np, pandas as pd
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential, Input, Model
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras import backend as K

import sys, datetime, os
# sys.path.append("./kslearn")
from kslearn import KerasModel, KerasClassifier, KerasRegressor
from kslearn import run_tb, display_log

In [ ]:
# test for KerasClassifier

rootpath = "./"+datetime.datetime.today().strftime("%Y%m%d_%H%M%S")

print("############## KerasClassifier ##############")
iris = datasets.load_iris()
x = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
#x_tr, x_te, y_tr, y_te = train_test_split(x, y, train_size  = 0.7)
num_classes = y.shape[1]

def mk_model_seq(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=4, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def mk_model_mo(activation="relu", optimizer="adam", out_dim=100):
    inputs = Input(shape=(4,)) 
    x = Dense(out_dim, kernel_initializer="normal")(inputs)
    x = Dense(num_classes, kernel_initializer="normal")(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def generator(x, y, batch_size=64):
    while True:
        for i in range(0, x.shape[0], batch_size):
            yield x[i:i+batch_size], y[i:i+batch_size]
            
activation = ["relu"]
optimizer = ["adam", "SGD"]
out_dim = [50, 100]
epochs = [3]
batch_size = [64]
param_grid = dict(activation=activation, 
                  optimizer=optimizer, 
                  out_dim=out_dim, 
                  epochs=epochs, 
                  batch_size=batch_size)

for model_type in ["seq", "model"]:
    if model_type == "seq":
        mk_model = mk_model_seq
    else:
        mk_model = mk_model_mo
        
    for y_type in ["cat", "1hot"]:
        if y_type == "cat":
            y_ = np.argmax(y, axis=1)
        else:
            y_ = y.copy()
        print("-----", model_type, y_type, "-----")

        print("# fit test")
        for reuse in [True, False]:
            print("## reuse:", str(reuse))
            model = KerasClassifier(build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "cls"), period=2, patience=10)

            print("1st fit")
            model.fit(x, y_, epochs=10)
            print(model.score(x,y_))

            print("2nd fit")
            model.fit(x, y_, epochs=10)
            print(model.score(x,y_))

        print(model.predict_generator(generator(x,y_), steps=1)[:1])
        print(model.predict(x)[:1])
        print(model.predict_proba(x)[:1])
        print(model.evaluate(x, y))
        
        print("# GS test")
        model = KerasClassifier(build_fn=mk_model, reuse=False, verbose=0, logroot_dir=os.path.join(rootpath, "cls_grid"), period=1, patience=10)
        grid = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True)
        grid.fit(x, y)
        print(pd.DataFrame(grid.cv_results_).iloc[0,0])

        if y_type == "1hot":
        #if True:
            print("# fit_gen test")
            for reuse in [True, False]:
                print("## reuse:", str(reuse))
                model = KerasClassifier(build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "cls_gen"), period=1, patience=10)

                print("1st fit")
                model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), classes=[0,1,2], epochs=3) 
                print(model.score(x,y_))

                print("2nd fit")
                model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), classes=[0,1,2], epochs=3) 
                print(model.score(x,y_))
            
            print(model.predict_generator(generator(x,y_), steps=1)[:1])
            print(model.evaluate_generator(generator(x,y_), steps=1)[:1])
            print(model.predict(x)[:1])
            print(model.predict_proba(x)[:1])
            print(model.evaluate(x, y))
            
            
            
            
# test for KerasRegressor
print("############## KerasRegressor ##############")
d = datasets.load_boston()
x = preprocessing.scale(d.data)
y = d.target
#x_tr, x_te, y_tr, y_te = train_test_split(x, y, train_size  = 0.7)

def mk_model_seq(activation="relu", optimizer="adam", out_dim=100):# iris
    model = Sequential()
    model.add(Dense(out_dim, input_dim=13, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(1))
    #model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

def mk_model_mo(activation="relu", optimizer="adam", out_dim=100):
    inputs = Input(shape=(13,)) 
    x = Dense(out_dim, kernel_initializer="normal")(inputs)
    x = Dense(1, kernel_initializer="normal")(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

def generator(x, y, batch_size=64):
    while True:
        for i in range(0, x.shape[0], batch_size):
            yield x[i:i+batch_size], y[i:i+batch_size]
            
activation = ["relu"]
optimizer = ["adam", "SGD"]
out_dim = [50, 100]
epochs = [3]
batch_size = [64]
param_grid = dict(activation=activation, 
                  optimizer=optimizer, 
                  out_dim=out_dim, 
                  epochs=epochs, 
                  batch_size=batch_size)

for model_type in ["seq", "model"]:
    if model_type == "seq":
        mk_model = mk_model_seq
    else:
        mk_model = mk_model_mo
        
#     for y_type in ["cat", "1hot"]:
#         if y_type == "cat":
#             y_ = np.argmax(y, axis=1)
#         else:
#             y_ = y.copy()
#         print("-----", model_type, y_type, "-----")
    print("-----", model_type, "-----")
    y_ = y.copy()

    print("# fit test")
    for reuse in [True, False]:
        print("## reuse:", str(reuse))
        model = KerasRegressor(build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "reg"), period=1, patience=10)

        print("1st fit")
        model.fit(x, y_, epochs=3)
        print(model.score(x,y_))

        print("2nd fit")
        model.fit(x, y_, epochs=3)
        print(model.score(x,y_))

    print(model.predict_generator(generator(x,y_), steps=1)[:1])
    print(model.predict(x)[:1])
    #print(model.predict_proba(x)[:1])
    print(model.evaluate(x, y))

    print("# GS test")
    model = KerasRegressor(build_fn=mk_model, reuse=False, verbose=0, logroot_dir=os.path.join(rootpath, "reg_gs"), period=1, patience=10)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True)
    grid.fit(x, y)
    print(pd.DataFrame(grid.cv_results_).iloc[0,0])

    #if y_type == "1hot":
    if True:
        print("# fit_gen test")
        for reuse in [True, False]:
            print("## reuse:", str(reuse))
            model = KerasRegressor(build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "reg_gen"), period=1, patience=10)

            print("1st fit")
            model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), epochs=3) 
            print(model.score(x,y_))

            print("2nd fit")
            model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), epochs=3) 
            print(model.score(x,y_))

        print(model.predict_generator(generator(x,y_), steps=1)[:1])
        print(model.evaluate_generator(generator(x,y_), steps=1)[:1])
        print(model.predict(x)[:1])
        #print(model.predict_proba(x)[:1])
        print(model.evaluate(x, y))
        
        
        
        
# test for KerasModel
print("############## KerasModel ##############")
iris = datasets.load_iris()
x = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
#x_tr, x_te, y_tr, y_te = train_test_split(x, y, train_size  = 0.7)
num_classes = y.shape[1]

def mk_model_seq(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=4, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def mk_model_mo(activation="relu", optimizer="adam", out_dim=100):
    inputs = Input(shape=(4,)) 
    x = Dense(out_dim, kernel_initializer="normal")(inputs)
    x = Dense(num_classes, kernel_initializer="normal")(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def generator(x, y, batch_size=64):
    while True:
        for i in range(0, x.shape[0], batch_size):
            yield x[i:i+batch_size], y[i:i+batch_size]
            
activation = ["relu"]
optimizer = ["adam", "SGD"]
out_dim = [50, 100]
epochs = [3]
batch_size = [64]
param_grid = dict(activation=activation, 
                  optimizer=optimizer, 
                  out_dim=out_dim, 
                  epochs=epochs, 
                  batch_size=batch_size)

for model_type in ["seq", "model"]:
    if model_type == "seq":
        mk_model = mk_model_seq
    else:
        mk_model = mk_model_mo
        
#     for y_type in ["cat", "1hot"]:
#         if y_type == "cat":
#             y_ = np.argmax(y, axis=1)
#         else:
#             y_ = y.copy()
#         print("-----", model_type, y_type, "-----")
    print("-----", model_type, "-----")
    y_ = y.copy()

    print("# fit test")
    for reuse in [True, False]:
        print("## reuse:", str(reuse))
        model = KerasModel(greater_is_better=True, build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "mo"), period=1, patience=10)

        print("1st fit")
        model.fit(x, y_, epochs=3)
        print(model.score(x,y_))

        print("2nd fit")
        model.fit(x, y_, epochs=3)
        print(model.score(x,y_))

    print(model.predict_generator(generator(x,y_), steps=1)[:1])
    print(model.predict(x)[:1])
    #print(model.predict_proba(x)[:1])
    print(model.evaluate(x, y))

    print("# GS test")
    model = KerasModel(greater_is_better=True, build_fn=mk_model, reuse=False, verbose=0, logroot_dir=os.path.join(rootpath, "mo_gs"), period=1, patience=10)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True)
    grid.fit(x, y)
    print(pd.DataFrame(grid.cv_results_).iloc[0,0])

    #if y_type == "1hot":
    if True:
        print("# fit_gen test")
        for reuse in [True, False]:
            print("## reuse:", str(reuse))
            model = KerasModel(greater_is_better=True, build_fn=mk_model, reuse=reuse, logroot_dir=os.path.join(rootpath, "mo_gen"), period=1, patience=10)

            print("1st fit")
            model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), epochs=3) 
            print(model.score(x,y_))

            print("2nd fit")
            model.fit_generator(generator(x,y_), steps_per_epoch=int(len(x)/64), epochs=3) 
            print(model.score(x,y_))

        print(model.predict_generator(generator(x,y_), steps=1)[:1])
        print(model.evaluate_generator(generator(x,y_), steps=1)[:1])
        print(model.predict(x)[:1])
        #print(model.predict_proba(x)[:1])
        print(model.evaluate(x, y))

In [ ]:
# test for display log
run_tb(rootpath, port=6006)

iris = datasets.load_iris()
x = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
#x_tr, x_te, y_tr, y_te = train_test_split(x, y, train_size  = 0.7)
num_classes = y.shape[1]

def mk_model(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=4, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=mk_model, logroot_dir=os.path.join(rootpath, "cls"), 
                        period=2, patience=10, tb_display_url="http://10.0.75.1:6006")
model.fit(x, y, epochs=10, verbose=1)

In [ ]:
# display_log(path=model.logroot_dir, tb_display_url="http://10.0.75.1:6006", cr_model_id=None)

In [ ]:
# Original KerasClassifier sample
from keras.wrappers.scikit_learn import KerasClassifier

iris = datasets.load_iris()
x = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
x_tr, x_te, y_tr, y_te = train_test_split(x, y, train_size  = 0.7)
num_classes = y_te.shape[1]

def iris_model(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=4, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


# search settings
activation = ["relu"]
optimizer = ["adam",]
out_dim = [100, 200]
nb_epoch = [10, 25]
batch_size = [5, 10]
param_grid = dict(activation=activation, 
                  optimizer=optimizer, 
                  out_dim=out_dim, 
                  nb_epoch=nb_epoch, 
                  batch_size=batch_size)

# run search
model = KerasClassifier(build_fn=iris_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x_tr, y_tr)
